<a href="https://colab.research.google.com/github/GrLau/Transformers-For-Stock-Price-Prediction/blob/main/TimesFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up environment for Google Research's Time Series Foundation Model -- TimesFM

In [ ]:
%env PYTHONPATH=
! wget [https://repo.anaconda.com/archive/Anaconda3-2024.02-1-Linux-x86_64.sh](https://repo.anaconda.com/archive/Anaconda3-2024.02-1-Linux-x86_64.sh)
! chmod +x Anaconda3-2024.02-1-Linux-x86_64.sh
! bash ./Anaconda3-2024.02-1-Linux-x86_64.sh -b -f -p /content/conda3
import os
path = '/content/conda3/bin:' + os.environ['PATH']
%env PATH=$path

!wget [https://github.com/google-research/timesfm/archive/refs/heads/master.zip](https://github.com/google-research/timesfm/archive/refs/heads/master.zip)
!unzip master.zip

!cd timesfm-master/

!conda env create --file=environment_cpu.yml

env: PYTHONPATH=
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: ` wget [https://repo.anaconda.com/archive/Anaconda3-2024.02-1-Linux-x86_64.sh](https://repo.anaconda.com/archive/Anaconda3-2024.02-1-Linux-x86_64.sh)'
chmod: cannot access 'Anaconda3-2024.02-1-Linux-x86_64.sh': No such file or directory
bash: ./Anaconda3-2024.02-1-Linux-x86_64.sh: No such file or directory
env: PATH=/content/conda3/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `wget [https://github.com/google-research/timesfm/archive/refs/heads/master.zip](https://github.com/google-research/timesfm/archive/refs/heads/master.zip)'
unzip:  cannot find or open master.zip, master.zip.zip or master.zip.ZIP.
/bin/bash: line 1: cd: timesfm-master/: No such file or directory
/bin/bash: line 1: cond

In [ ]:
import sys
# maybe only need this the first time we run this notebook
sys.path.append('/usr/local/lib/python3.10/site-packages')

print("Python version")  # These lines check to make sure it's working
print(sys.version) # These lines check to make sure it's working


Python version
3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
!pip install --upgrade pip
!pip install -e .
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
#Restart session when prompted to after running this cell
!pip install timesfm

  Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (787 bytes)
Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.27.2
    Uninstalling protobuf-5.27.2:
      Successfully uninstalled protobuf-5.27.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 3.19.6 which is incompatible.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.


In [ ]:
import datetime
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import timesfm
from timesfm import TimesFm
from huggingface_hub import login

# Get the stock data
start = datetime.date(2020, 1, 1)
end = datetime.date(2024, 1, 1)
current_date = datetime.date(2024, 6, 14)
#current_date = datetime.date.today()
codelist = ["AAPL"]

# Add the download data part of the error retry mechanism
for retry in range(3):  # Try downloading up to 3 times
    try:
        data2 = yf.download(codelist, start=start, end=end) #From start to 2024
        data3 = yf.download(codelist, start=end, end=current_date) #From 2024 to Present

        # Data preprocessing
        data2 = data2['Close'].dropna()  # Use the Closing price to remove missing values
        data3 = data3['Close'].dropna()
        if not data2.empty:
            break  # Successfully download and process data, jump out of the loop
    except Exception as e:
        print(f"Download failed, {retry+1}th attempt. Error：{e}")
        if retry < 2:  # wait before final attempt
            time.sleep(5)  # Wait 5 seconds and try again

if data2.empty:
    raise ValueError("The data is empty. Please change the time interval and try again or check the network connection.")

context_len = 32  # Set the context length
horizon_len = len(data3)  # Set the length of the forecast period, currently from 2024 to Present

if len(data2) < context_len:
    raise ValueError(f"data length is less than context length（{context_len}）")

context_data = data2[-context_len:]  # Use the data of the last 512 days as context

# Intialize and import TimesFM model
tfm = TimesFm(
    context_len=context_len,
    horizon_len=horizon_len,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend='cpu',  # Modify here, change 'gpu' to 'cpu'
)

# Log in to Hugging Face Hub
login("***")#Please replace it with your own

tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

# Prepare data
forecast_input = [context_data.values]
frequency_input = [0]  #Set data frequency（0 is high frequency data）

# Run predictions
point_forecast, experimental_quantile_forecast = tfm.forecast(
    forecast_input,
    freq=frequency_input,
)

# View predictions results
forecast_dates = pd.date_range(start=pd.to_datetime(data2.index[-1]) + pd.Timedelta(days=1), periods=horizon_len, freq='B')
forecast_series = pd.Series(point_forecast[0], index=forecast_dates)

plt.figure(figsize=(14, 7))
plt.plot(data2.index, data2.values, label="Actual Prices")
plt.plot(forecast_series.index, forecast_series.values, label="Forecasted Prices")
plt.plot(data3.index, data3.values, label="Actual Prices (2024-Present)")
plt.xlabel("Date", fontsize=20)
plt.ylabel("Price", fontsize=20)
plt.title((codelist[0] + " Forecasted Prices"), fontsize=20)  # Add a title, optional
plt.legend()




# Save the chart to a file
plt.savefig('TimesFm_'+codelist[0]+'_forecast.png')  # The file name can be customized as needed

# Note: If you run this code in a non-GUI environment (such as a server), make sure to set the backend to 'Agg' before
plt.switch_backend('Agg')

# Usually after saving the chart, in order to avoid opening the image window in a non-GUI environment, you can explicitly close the figure
plt.close() #View graph in notebook files

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

...

(…)nts/checkpoint_1100000/metadata/metadata:   0%|          | 0.00/43.9k [00:00<?, ?B/s]

Constructing model weights.


Constructed model weights in 3.58 seconds.
Restoring checkpoint from /root/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 1.41 seconds.
Jitting decoding.
Jitted decoding in 31.57 seconds.


In [ ]:
from sklearn.metrics import mean_absolute_error


# Calculate MAE
mae = mean_absolute_error(forecast_series, data3)
print("TimesFm MAE:", mae)

TimesFm MAE: 10.852589925130209
